In [79]:
# Import the dependencies.

import pandas as pd
import numpy as np
from citipy import citipy
from datetime import datetime
import requests

# Import the API key.
from config import weather_api_key

In [80]:
# Randomly generate 2000 latitudes and longitudes

lats = np.random.uniform(low = -90, high = 90, size = 2000)
lngs = np.random.uniform(low = -180, high = 180, size = 2000)

# Combine latitudes and longitudes

lats_lngs = zip(lats,lngs)

# Store object in coordinates list

coordinates = list(lats_lngs)

coordinates

[(18.14109729928707, -166.26240235893917),
 (43.83194137008087, 100.51516969418162),
 (-4.881783479219038, 67.14360873232172),
 (44.59874765817301, -76.92517765440643),
 (71.74097188178257, -18.0184933498918),
 (-26.4280510707789, 41.86132027776026),
 (89.42864370517572, 133.2992321847251),
 (-15.373492185100773, 39.37658491139058),
 (5.6698556107209015, 63.73949242673626),
 (-88.03259662230852, 139.5760279887374),
 (66.49916942557783, -93.99367568993023),
 (69.7391182239297, -40.319175804411174),
 (-15.901937887713274, -8.544348514109743),
 (-48.20990853196017, 20.355221163614715),
 (69.45950818482194, -159.10669025970793),
 (-89.32301809390327, -118.87439473589052),
 (35.419127487262216, 179.04562252391656),
 (-78.88892422392472, -159.18486833490522),
 (76.39679820986981, 100.84061503997378),
 (-47.21232198465268, -132.00927955014856),
 (-27.390042620068677, 103.04915463234971),
 (41.32829301920614, -173.86466650485232),
 (33.75199078555694, -23.80989976199058),
 (71.32474885107544, 

In [81]:
# Iterating through coordinates list and finding nearest city using citipy module

# Step 1: Create an empty list to hold cities

cities = []

# Step 2: Iterate through coordinates and append cities list

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)

len(cities)

727

In [82]:
# Create preliminary dictionary and convert it to a DataFrame

city_weather_dct = {"City" : cities,
                   "Country" : "",
                   "Lat" : "",
                   "Long" : "",
                   "Max Temp" : "",
                   "Humidity" : "",
                   "Cloudiness" : "",
                   "Wind Speed" : "",
                   "Current Description" : ""}

city_weather_df = pd.DataFrame(city_weather_dct)

# Test DataFrame

city_weather_df

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,kapaa,,,,,,,,
1,hovd,,,,,,,,
2,hithadhoo,,,,,,,,
3,napanee,,,,,,,,
4,illoqqortoormiut,,,,,,,,
...,...,...,...,...,...,...,...,...,...
722,nosy varika,,,,,,,,
723,bafra,,,,,,,,
724,guayaramerin,,,,,,,,
725,ewa beach,,,,,,,,


In [83]:
# Create a params dict that will be updated with new city each iteration

params = {"appid": weather_api_key}

# Loop through the city_weather_df and run a search for each city

for index, row in city_weather_df.iterrows():
    

    city = row["City"]

    # update address key value
    
    params["q"] = f"{city}"
    
    # Add base URL 
    
    base_url = "https://api.openweathermap.org/data/2.5/weather?units=imperial"

    # make request
    
    weather_data = requests.get(base_url, params=params)
    
    # convert to json
    weather_data = weather_data.json()

    try:
    
        city_weather_df.loc[index, "Country"] = weather_data["sys"]["country"]
        city_weather_df.loc[index, "Lat"] = weather_data["coord"]["lat"]
        city_weather_df.loc[index, "Long"] = weather_data["coord"]["lon"]
        city_weather_df.loc[index, "Max Temp"] = weather_data["main"]["temp_max"]
        city_weather_df.loc[index, "Humidity"] = weather_data["main"]["humidity"]
        city_weather_df.loc[index, "Cloudiness"] = weather_data["clouds"]["all"]
        city_weather_df.loc[index, "Wind Speed"] = weather_data["wind"]["speed"]
        city_weather_df.loc[index, "Current Description"] = weather_data["weather"][0]["description"]
        
    except(KeyError, IndexError):
        
        city_weather_df.loc[index, "Country"] = np.nan
        city_weather_df.loc[index, "Lat"] = np.nan
        city_weather_df.loc[index, "Long"] = np.nan
        city_weather_df.loc[index, "Max Temp"] = np.nan
        city_weather_df.loc[index, "Humidity"] = np.nan
        city_weather_df.loc[index, "Cloudiness"] = np.nan
        city_weather_df.loc[index, "Wind Speed"] = np.nan
        city_weather_df.loc[index, "Current Description"] = np.nan
        
        print("City not found... skipping")
    

City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
City not found... skipping
C

In [85]:
# Find Null Values

city_weather_df.count()

City                   727
Country                677
Lat                    677
Long                   677
Max Temp               677
Humidity               677
Cloudiness             677
Wind Speed             677
Current Description    677
dtype: int64

In [87]:
# Remove NA's from the DataFrame

city_weather_df = city_weather_df.dropna()

# Preview the DataFrame

city_weather_df

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,kapaa,US,22.0752,-159.319,77.32,78,75,11.5,moderate rain
1,hovd,MN,48.0056,91.6419,47.1,37,4,7.92,clear sky
2,hithadhoo,MV,-0.6,73.0833,81.82,79,72,18.9,light rain
3,napanee,CA,44.2501,-76.9494,69.84,97,100,8.63,moderate rain
5,ampanihy,MG,-24.7,44.75,68.27,72,37,17.13,scattered clouds
...,...,...,...,...,...,...,...,...,...
722,nosy varika,MG,-20.5833,48.5333,69.57,76,4,3.94,clear sky
723,bafra,TR,41.5678,35.9069,56.89,88,100,9.31,light rain
724,guayaramerin,BO,-10.8368,-65.3614,89.65,51,0,5.75,clear sky
725,ewa beach,US,21.3156,-158.0072,76.95,72,90,5.75,overcast clouds


In [88]:
# Perform Count Function to test dropna()

city_weather_df.count()

City                   677
Country                677
Lat                    677
Long                   677
Max Temp               677
Humidity               677
Cloudiness             677
Wind Speed             677
Current Description    677
dtype: int64

In [89]:
# Format the DataFrame before Exporting

# Capitalize the City for Formatting

city_weather_df["City"] = city_weather_df["City"].str.title()

# Reset the Index

city_weather_df.reset_index(drop = True, inplace = True)

C:\Users\lsmav\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.319,77.32,78,75,11.5,moderate rain
1,Hovd,MN,48.0056,91.6419,47.1,37,4,7.92,clear sky
2,Hithadhoo,MV,-0.6,73.0833,81.82,79,72,18.9,light rain
3,Napanee,CA,44.2501,-76.9494,69.84,97,100,8.63,moderate rain
4,Ampanihy,MG,-24.7,44.75,68.27,72,37,17.13,scattered clouds
...,...,...,...,...,...,...,...,...,...
672,Nosy Varika,MG,-20.5833,48.5333,69.57,76,4,3.94,clear sky
673,Bafra,TR,41.5678,35.9069,56.89,88,100,9.31,light rain
674,Guayaramerin,BO,-10.8368,-65.3614,89.65,51,0,5.75,clear sky
675,Ewa Beach,US,21.3156,-158.0072,76.95,72,90,5.75,overcast clouds


In [90]:
# Preview the DataFrame Before Exporting

city_weather_df

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.319,77.32,78,75,11.5,moderate rain
1,Hovd,MN,48.0056,91.6419,47.1,37,4,7.92,clear sky
2,Hithadhoo,MV,-0.6,73.0833,81.82,79,72,18.9,light rain
3,Napanee,CA,44.2501,-76.9494,69.84,97,100,8.63,moderate rain
4,Ampanihy,MG,-24.7,44.75,68.27,72,37,17.13,scattered clouds
...,...,...,...,...,...,...,...,...,...
672,Nosy Varika,MG,-20.5833,48.5333,69.57,76,4,3.94,clear sky
673,Bafra,TR,41.5678,35.9069,56.89,88,100,9.31,light rain
674,Guayaramerin,BO,-10.8368,-65.3614,89.65,51,0,5.75,clear sky
675,Ewa Beach,US,21.3156,-158.0072,76.95,72,90,5.75,overcast clouds


In [91]:
# Create the output file (CSV).

output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.

city_weather_df.to_csv(output_data_file, index_label="City_ID")